# CNN Visualization
1. Activation maps
2. Filter using gradient ascent
3. Deep dream using gradient ascent

In [1]:
%env CUDA_VISIBLE_DEVICES = ""

env: CUDA_VISIBLE_DEVICES=""


In [2]:
import os 
import tensorflow as tf 
import tensorflow.keras as keras
# tf.config.gpu.set_per_process_memory_growth(True)
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
tf.compat.v1.disable_eager_execution()
sess = tf.compat.v1.Session()

Use a pretrained model like VGG16 and VGG19 trained on ImageNet to do the task.

In [3]:
vgg19 = keras.applications.VGG19(include_top=False, input_shape=(224, 224, 3), weights='imagenet')

In [4]:
vgg19.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [5]:
# version = 1
# export_path = './model/{}'.format(version)
# builder = tf.compat.v1.saved_model.builder.SavedModelBuilder(export_path)

# ############### Build the map for signature_def ################
# inputs = tf.compat.v1.saved_model.build_tensor_info(vgg19.input)

# # map method name to tensor info
# map_dict = []
# for l in vgg19.layers[1:3]:
#     l_ts = tf.cast(l.output, tf.int32)
#     l_ts = l_ts[:, :, :, :4]
#     map_dict.append((l.name, tf.compat.v1.saved_model.build_tensor_info(l_ts)))
# map_dict = dict(map_dict)

# # signature method name must be CLASSIFY_METHOD_NAME, PREDICT_METHOD_NAME or REGRESS_METHOD_NAME, 
# # they have different request format. please refer to https://www.tensorflow.org/tfx/serving/signature_defs 
# # and https://www.tensorflow.org/tfx/serving/api_rest/    
# signature = (
#     tf.compat.v1.saved_model.signature_def_utils.build_signature_def(
#         inputs={'input': inputs},
#         outputs=map_dict,
#         method_name=tf.compat.v1.saved_model.signature_constants.PREDICT_METHOD_NAME 
#     )
# )


# # builder.add_meta_graph_and_variables(
# #     tf.compat.v1.keras.backend.get_session(), [tf.compat.v1.saved_model.tag_constants.SERVING],
# #     signature_def_map={'activation':signature,},
# #     strip_default_attrs=True
# # )
# # builder.save()

# Deconv

In [6]:
def deconvnet_model(model, layer_name):
    deconv_layers = []
    for i in range(len(model.layers)):
        if isinstance(model.layers[i], keras.layers.Conv2D):
            weights = model.layers[i].get_weights()[0]
            weights_h, weights_w, inp_filters, out_filters = weights.shape
            deconv_layer = keras.layers.Conv2DTranspose(inp_filters, (weights_h, weights_w),
                                                  input_shape=(model.layers[i].output_shape[1:]),
                                                  strides=model.layers[i].get_config()['strides'],
                                                  padding=model.layers[i].get_config()['padding'],
                                                  activation=model.layers[i].get_config()['activation'],
                                                  kernel_initializer=tf.constant_initializer(weights), 
                                                  bias_initializer=tf.zeros_initializer())
            deconv_layers.append(deconv_layer)
        elif isinstance(model.layers[i], keras.layers.MaxPooling2D):
            deconv_layer = keras.layers.UpSampling2D(model.layers[i].get_config()['pool_size'], interpolation="bilinear")
            deconv_layers.append(deconv_layer)
        
        elif isinstance(model.layers[i], keras.layers.AveragePooling2D):
            deconv_layer = keras.layers.UpSampling2D(model.layers[i].get_config()['pool_size'], interpolation="bilinear")
            deconv_layers.append(deconv_layer)
            
        if model.layers[i].name == layer_name:
            break
    deconv_model = tf.keras.Sequential(deconv_layers[::-1])
    return deconv_model

# visualize the filter activation using deconvolution model.
def post_process(img):
    img -= tf.reduce_min(img, axis=[1,2,3], keepdims=True)
    img *= 1.0 / (tf.reduce_max(img, axis=[1,2,3], keepdims=True) + 1e-8)
    img = tf.cast(img * 255, tf.int32)
    return img

def visualization_using_deconvolution(orig_model, deconv_model, vis_image, feature_to_visualize):
    vis_image = tf.image.resize(vis_image, (224, 224))

    vis_image = orig_model(vis_image)
    feature_map = vis_image[:, :, :, feature_to_visualize:feature_to_visualize+1] # shape [batch_size, width, height, 1]
    paddings = [[0,0],
                [0,0],
                [0,0],
                [feature_to_visualize,vis_image.shape[-1]-feature_to_visualize-1]]
    output = tf.pad(feature_map, paddings, 'CONSTANT')
    deconv_output = deconv_model(output)
    return deconv_output

In [7]:
vis_layer = 'block4_conv1'
target_output = vgg19.get_layer(vis_layer).output 
vgg19_submodel = tf.keras.Model(vgg19.input, target_output)
vgg19_deconv = deconvnet_model(vgg19, vis_layer)

In [8]:
vis_outputs = []
img = tf.compat.v1.placeholder(dtype=tf.float32, shape=[1, 224, 224, 3])
for idx in range(16):
    vis_output = visualization_using_deconvolution(vgg19_submodel, vgg19_deconv, img, idx)
    vis_outputs.append(post_process(vis_output))
layer_outputs = tf.stack(vis_outputs)

In [9]:
layer_outputs

<tf.Tensor 'stack:0' shape=(16, 1, 224, 224, 3) dtype=int32>

In [9]:
version = 1
export_path = './model/{}'.format(version)
builder = tf.compat.v1.saved_model.builder.SavedModelBuilder(export_path)

############### Build the map for signature_def ################
inputs = tf.compat.v1.saved_model.build_tensor_info(img)

# map method name to tensor info
map_dict = []
# img = tf.cast(img, tf.int32)
map_dict.append((layer_outputs.name, tf.compat.v1.saved_model.build_tensor_info(layer_outputs)))
map_dict = dict(map_dict)

# signature method name must be CLASSIFY_METHOD_NAME, PREDICT_METHOD_NAME or REGRESS_METHOD_NAME, 
# they have different request format. please refer to https://www.tensorflow.org/tfx/serving/signature_defs 
# and https://www.tensorflow.org/tfx/serving/api_rest/    
signature2 = (
    tf.compat.v1.saved_model.signature_def_utils.build_signature_def(
        inputs={'input': inputs},
        outputs=map_dict,
        method_name=tf.compat.v1.saved_model.signature_constants.PREDICT_METHOD_NAME 
    )
)

# I don't know why, but can't use same tag here, or their key mix?
builder.add_meta_graph_and_variables(
    tf.compat.v1.keras.backend.get_session(),
    [tf.compat.v1.saved_model.tag_constants.SERVING],
    signature_def_map={'deconv':signature2,},
    strip_default_attrs=True
)
# builder.add_meta_graph(['deconv'],
#     signature_def_map={'deconv':signature2,},
#     strip_default_attrs=True
# )
builder.save()

W0827 17:28:31.245250 140215473108736 deprecation.py:323] From <ipython-input-9-e6af897fbb12>:6: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


b'./model/1/saved_model.pb'

In [10]:
map_dict

{'stack:0': name: "stack:0"
 dtype: DT_INT32
 tensor_shape {
   dim {
     size: 16
   }
   dim {
     size: 1
   }
   dim {
     size: 224
   }
   dim {
     size: 224
   }
   dim {
     size: 3
   }
 }}

In [1]:
!saved_model_cli show --dir ./model/1 --all

Traceback (most recent call last):
  File "/home/kjliu/.local/lib/python3.5/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/kjliu/.local/lib/python3.5/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/kjliu/.local/lib/python3.5/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/usr/lib/python3.5/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/usr/lib/python3.5/imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: libcublas.so.9.0: cannot open shared object file: No such file or directory

During handling of the above exception, another exception occurred:

Traceback (most recent call last